In [2]:
from pyspark.sql import SparkSession
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
spark = SparkSession.builder \
    .appName("Lab 4") \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/08/21 10:29:42 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## Q1

In [4]:
df = spark.read.json('./movies 1.json')
df.printSchema()
df.show()

root
 |-- helpfulness: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- profile_name: string (nullable = true)
 |-- review: string (nullable = true)
 |-- score: double (nullable = true)
 |-- summary: string (nullable = true)
 |-- time: long (nullable = true)
 |-- user_id: string (nullable = true)

+-----------+----------+--------------------+--------------------+-----+--------------------+----------+--------------+
|helpfulness|product_id|        profile_name|              review|score|             summary|      time|       user_id|
+-----------+----------+--------------------+--------------------+-----+--------------------+----------+--------------+
|        7/7|B003AI2VGA|Brian E. Erland "...|Synopsis: On the ...|  3.0|"There Is So Much...|1182729600|A141HP4LYPWMSR|
|        4/4|B003AI2VGA|          Grady Harp|THE VIRGIN OF JUA...|  3.0|Worthwhile and Im...|1181952000|A328S9RN3U5M68|
|       8/10|B003AI2VGA|Chrissy K. McVay ...|The scenes in thi...|  5.0|This m

## Q3

In [5]:
user_indexer = StringIndexer(inputCol="user_id", outputCol="userIndex",handleInvalid="keep")
df = user_indexer.fit(df).transform(df)


product_indexer = StringIndexer(inputCol="product_id", outputCol="productIndex", )
df = product_indexer.fit(df).transform(df)


In [6]:
df = df.select(
    col("userIndex").alias("userId"),
    col("productIndex").alias("itemId"),
    col("score").alias("rating")
)

## Q2

In [7]:
(training_data, test_data) = df.randomSplit([0.6, 0.4])

## Q3

In [8]:
als = ALS(
    maxIter=25,
    regParam=0.01,
    userCol="userId",
    itemCol="itemId",
    ratingCol="rating",
    coldStartStrategy="drop"
)
model = als.fit(training_data)
predictions = model.transform(test_data)

24/08/21 10:29:46 WARN DAGScheduler: Broadcasting large task binary with size 1623.3 KiB
24/08/21 10:29:46 WARN DAGScheduler: Broadcasting large task binary with size 1625.7 KiB
24/08/21 10:29:46 WARN DAGScheduler: Broadcasting large task binary with size 1627.2 KiB
24/08/21 10:29:46 WARN DAGScheduler: Broadcasting large task binary with size 1628.5 KiB
24/08/21 10:29:47 WARN DAGScheduler: Broadcasting large task binary with size 1627.4 KiB
24/08/21 10:29:47 WARN DAGScheduler: Broadcasting large task binary with size 1628.7 KiB
24/08/21 10:29:47 WARN DAGScheduler: Broadcasting large task binary with size 1629.5 KiB
24/08/21 10:29:47 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.VectorBLAS
24/08/21 10:29:47 WARN DAGScheduler: Broadcasting large task binary with size 1632.5 KiB
24/08/21 10:29:47 WARN DAGScheduler: Broadcasting large task binary with size 1633.9 KiB
24/08/21 10:29:48 WARN DAGScheduler: Broadcasting large task binary with size 1635.3 KiB


## Q4

In [9]:
rmse_evaluator = RegressionEvaluator(
    metricName="rmse",
    labelCol="rating",
    predictionCol="prediction"
)

# Calculate RMSE
rmse = rmse_evaluator.evaluate(predictions)
print(f"Root Mean Squared Error (RMSE) = {rmse}")

24/08/21 10:29:54 WARN DAGScheduler: Broadcasting large task binary with size 1607.7 KiB
24/08/21 10:29:54 WARN DAGScheduler: Broadcasting large task binary with size 1709.1 KiB
24/08/21 10:29:54 WARN DAGScheduler: Broadcasting large task binary with size 1707.7 KiB


Root Mean Squared Error (RMSE) = 3.469711533079103


24/08/21 10:29:54 WARN DAGScheduler: Broadcasting large task binary with size 1756.6 KiB


In [10]:
mae_evaluator = RegressionEvaluator(
    metricName="mae",
    labelCol="rating",
    predictionCol="prediction"
)

# Calculate MAE
mae = mae_evaluator.evaluate(predictions)
print(f"Mean Absolute Error (MAE) = {mae}")

24/08/21 10:29:54 WARN DAGScheduler: Broadcasting large task binary with size 1607.7 KiB
24/08/21 10:29:55 WARN DAGScheduler: Broadcasting large task binary with size 1709.1 KiB
24/08/21 10:29:55 WARN DAGScheduler: Broadcasting large task binary with size 1707.7 KiB


Mean Absolute Error (MAE) = 2.574794666670237


24/08/21 10:29:55 WARN DAGScheduler: Broadcasting large task binary with size 1756.6 KiB


In [11]:
predictions_with_actuals = predictions.select(
    col("userId"),
    col("itemId"),
    col("rating"),
    col("prediction")
)

# Show the first few rows of predictions with actual ratings
predictions_with_actuals.show(10)

24/08/21 10:29:55 WARN DAGScheduler: Broadcasting large task binary with size 1608.0 KiB
24/08/21 10:29:55 WARN DAGScheduler: Broadcasting large task binary with size 1709.1 KiB
24/08/21 10:29:55 WARN DAGScheduler: Broadcasting large task binary with size 1707.7 KiB


+------+------+------+----------+
|userId|itemId|rating|prediction|
+------+------+------+----------+
|3749.0|  21.0|   3.0| 2.9961803|
| 540.0|   7.0|   5.0| 1.7783462|
|1143.0|  85.0|   4.0| 1.3242304|
|1339.0| 680.0|   4.0|-0.7353947|
|1903.0|   7.0|   5.0|0.44551587|
|4294.0|  21.0|   5.0| 4.9936337|
|5433.0|  37.0|   5.0|0.86391574|
|3306.0|  37.0|   5.0|0.44908813|
|1005.0|   7.0|   1.0| 0.9986076|
|1005.0|   7.0|   1.0| 0.9986076|
+------+------+------+----------+
only showing top 10 rows

